
<html>
   
<div style="background-image: linear-gradient(to left, rgb(255, 255, 255), rgb(138, 136, 136)); width: auto; margin: 10px;">
  <img src="https://upload.wikimedia.org/wikipedia/en/thumb/f/fd/University_of_Tehran_logo.svg/225px-University_of_Tehran_logo.svg.png" width=100px width=auto style="padding:10px; vertical-align: center;">
 
</div>
   
<div   style:"text-align: center; background-image: linear-gradient(to left, rgb(255, 255, 255), rgb( 219, 204, 245  ));width: 400px; height: 30px; ">
<h1 style="font-family: Georgia; color: black; text-align: center; ">Course: AI </h1>

</div>
    <div   style:"border: 3px solid green;text-align: center; ">
<h1 style="font-family: Georgia; color: black; text-align: center; ">Project1: Genetics </h1>

</div>

   <div>    
<h1 style="font-family: Georgia; color: black; text-align: center; font-size:15px;">Mohammad Amanlou- sid:810100084 </h1>

</div>
   

</html>

### The purpose of the project:
The main goal of this project is to do a curve fitting problem using "Genetic Algorithms" so that by receiving a number of points, we can fit a function with a certain degree that is the closest and the least far from these points.


### General description of the project and definition of the problem
Genetic algorithms are generally used in problems with a large state space; These algorithms are sampled from
population and the composition and change of people and their assessment and try to pass the answers from generation to generation
improve and reach the desired answer
In this problem with the help of genetic algorithms we
must take into account the points that you
given, define the coefficients of a polynomial expression. The goal is based on points
Given, the relationship between the variable (x) and the variable (y) should be determined.
The inputs of the problem will include the degree of the desired polynomial, the coefficient range and the number of points on that polynomial


In [57]:
import pandas as pd
import numpy as np
import random

In [58]:
dots = [(0,1) , (1,0) , (2,-5) , (-1,-8)]
dots = [(-2, 1) , (-1, -18) , (0, -1) , (1, 10) , (2, 237) , (3, 2486) , (-3,1202)]
#dots = [(0,1) , (-1,-4) , (2,-31) , (-2,-103) , (1,5)]

First we need to create a population of chromosomes.
In this problem, each chromosome is a list of equation coefficients and each gene of this chromosome is one of these coefficients.
We create it using the randint function. And we receive the lower and upper limits of the coefficients along with the number of population members from the user.
Finally, the data is randomly generated and improved at each step.


In [59]:
def make_population(degree , pop_num , low_lim , up_lim):
    Random = np.random.randint(low=low_lim, high=up_lim + 1, size=(pop_num ,  degree + 1))
    df = pd.DataFrame(Random)
    return df

Then we have to choose an index as fitness. In this issue, we define fitness in such a way that the lower the better. And we consider it as the SSE value, or the sum of squares errors, which is calculated from the following formula.(Residual sum of squares)
$$ {\displaystyle \operatorname {SSE} =\sum _{i=1}^{n}(y_{i}-f(x_{i}))^{2}} $$

In [60]:
def calculate_sse(dots, coefficients):
    for index, row in coefficients.iterrows():
        sse = 0
        for dot in dots:
            x, y = dot
            y_pred = 0
            for degree in list(coefficients.columns)[0:-1]:
                coeff = row[degree]
                y_pred += coeff * (x ** int(degree))
            sse += (y - y_pred) ** 2
        coefficients.at[index,'SSE']=sse
        #print(coefficients)
    return coefficients

One of the possible solutions is to take happier chromosomes to the next generation without crossover and mutation, but here we refrained from doing this due to the method considered for crossover, but its function was implemented so that if You need to be able to use this feature as well


In [61]:
def elites(pop):
    population.sort_values("SSE")[:int(0.2*len(population))].sample(frac=1)

According to the conditions of the problem and the necessity that the number of members of each generation should remain constant, in the selection stage, as the first stage, we select the half of each generation that has a higher fitness level, and we go to the crossover stage. Generation The number of members of the entire generation remains constant. The issue that is raised is because the number of members of the generation must be even in order to create children, if the number of members of our generation is odd, we add the best chromosome to the end of the generation and finally we have to randomize the placement order and It makes the solution converge later.


In [62]:
def selection(population):
    population = population.sort_values("SSE")[:int(0.5*len(population))].sample(frac=1)
    if(len(population)%2 != 0):
        duplicated_row = population.iloc[[0]]
        population = population.append(duplicated_row, ignore_index=True)
    return population

Various crossover methods have been implemented below, which are created in the 1-point method by choosing a point and moving all the genes after that point with the other parent.


In [63]:
def one_point_crossover(pop , PCross):
    if(len(pop)%2 != 0):
        duplicated_row = pop.iloc[[0]]
        pop = pop.append(duplicated_row, ignore_index=True)
    childs = pop.copy(deep = True)
    pc = random.uniform(0, 1)
    if(pc > PCross/100):
        crosover_point = random.randint(1 , int(list(childs.columns)[-2]) )
        for c in range(0 , len(childs) , 2):
            for i in range(crosover_point , len(childs.columns)):
                childs.iloc[c , i],childs.iloc[ c+1 , i] = childs.iloc[ c+1 , i],childs.iloc[c , i]
    return childs

In another method, we do this by using n points, we consider a probability Pc for the points to be moved or not, and we move that gene according to that probability. We do this on all genes for two parents


In [64]:
def n_point_crossover(pop , PCross):
    childs = pop.copy(deep = True)
    for c in range(0 , len(childs) , 2):
        for i in range(0 , len(childs.columns)-1):
            pc = random.uniform(0, 1)
            if(pc > (100-PCross)/100):
                childs.iloc[c , i],childs.iloc[ c+1 , i] = childs.iloc[ c+1 , i],childs.iloc[c , i]
    return childs

In the next step, we have to do mutation. At this stage, by receiving a probability from the user as Pm, we change each gene with that probability to try to accidentally improve the result or to avoid getting stuck in relative extremes.


In [65]:
def mutation(population , low_co , high_co , degree , PMut , points):
    for index, row in population.iterrows():
        for d in range(degree):
            pm = random.uniform(0, 1)
            if (pm > (100-PMut)/100):
                population.at[index,'SSE'] = random.sample(range(low_co,high_co+1), 1)[0]
    return population

Finally, we consider all the previous steps and create a top-level module to implement the genetic algorithm, which performs all the above steps in the number of epochs.
Also, keeping in mind that if the error rate reaches zero, we can be sure that the problem has been solved correctly, we also add an early drop section to stop the process if we get the correct answer.

In [66]:
def top_level(degree , pop_num , low_lim , up_lim , PCross , PMut , dots , epochs):
    population = make_population(degree , pop_num , low_lim , up_lim)
    population["SSE"] = 0
    population = calculate_sse(dots, population)
    for e in range(epochs):
        parents = selection(population)
        children = n_point_crossover(parents , PCross)
        population = pd.concat([parents , children] , axis=0 , ignore_index=True)
        population = mutation(population , low_lim , up_lim , degree , PMut , dots )
        population["SSE"] = 0
        population = calculate_sse(dots, population)
        min_sse = min(population["SSE"])
        print("epoch num " + str(e) + " SSE = " + str(min_sse))
        if(min_sse == 0): #early dropping
            break
    return population.sort_values("SSE").iloc[0,:]

Finally, if we want to explain the problem solving process step by step. We should mention the following.

1. **Initialization**: The `make_population` function is called to initialize a population of individuals, where each individual represents a possible solution to the curve fitting problem. The population size, degree of the polynomial, and the lower and upper limits for the coefficients are passed as parameters.

2. **Fitness Calculation**: The `calculate_sse` function is called to calculate the Sum of Squared Errors (SSE) for each individual in the population. The SSE is used as a measure of fitness, with lower values indicating better fit.

3. **Selection**: In each epoch (iteration), the `selection` function is called to select parents from the current population based on their fitness.

4. **Crossover**: The `n_point_crossover` function is called to perform crossover between pairs of parents, producing children that inherit traits from both parents.

5. **Mutation**: The `mutation` function is called to introduce small random changes in the children, helping to maintain diversity in the population and prevent premature convergence.

6. **Evaluation**: After mutation, the SSE for each individual in the population (both parents and children) is recalculated.

7. **Termination Check**: The algorithm checks if the minimum SSE has reached 0, in which case it terminates early. Otherwise, it proceeds to the next epoch.

8. **Result**: After all epochs have been completed (or early termination has been triggered), the individual with the lowest SSE is returned as the best solution.


In [56]:
top_level(6 , 1000 , -10 , 10 , 50 , 10 , dots , 100)

epoch num 0 SSE = 44463
epoch num 1 SSE = 40264
epoch num 2 SSE = 34436
epoch num 3 SSE = 3224
epoch num 4 SSE = 3224
epoch num 5 SSE = 3224
epoch num 6 SSE = 3224
epoch num 7 SSE = 1852
epoch num 8 SSE = 1852
epoch num 9 SSE = 1267
epoch num 10 SSE = 1259
epoch num 11 SSE = 1259
epoch num 12 SSE = 1259
epoch num 13 SSE = 1259
epoch num 14 SSE = 552
epoch num 15 SSE = 279
epoch num 16 SSE = 279
epoch num 17 SSE = 112
epoch num 18 SSE = 112
epoch num 19 SSE = 112
epoch num 20 SSE = 112
epoch num 21 SSE = 91
epoch num 22 SSE = 56
epoch num 23 SSE = 56
epoch num 24 SSE = 35
epoch num 25 SSE = 35
epoch num 26 SSE = 35
epoch num 27 SSE = 35
epoch num 28 SSE = 35
epoch num 29 SSE = 7
epoch num 30 SSE = 7
epoch num 31 SSE = 7
epoch num 32 SSE = 7
epoch num 33 SSE = 7
epoch num 34 SSE = 7
epoch num 35 SSE = 7
epoch num 36 SSE = 0


0     -1
1      7
2     -2
3      5
4     -4
5      2
6      3
SSE    0
Name: 606, dtype: int64

As we know, in genetic algorithms, due to the use of randomness in all steps, every time the algorithm is executed, it is possible to reach the solution in more or less steps or even get stuck at a relative extreme point and not reach the exact solution. As we can see in the test above, this method has been able to predict a function of degree 6 without any error using 7 points.

It should be noted that if the error value is not zero and, for example, the SSE value for 7 points is 28, this means that the prediction made is only 2 units away from the actual value of the function at that point. Why so:

$ {\displaystyle \operatorname {28} =\sum _{i=1}^{7}(y_{i}-f(x_{i}))^{2}} $

As a result, the average value of $(y_{i}-f(x_{i}))^{2}$ is equal to 4 and average value of $(y_{i}-f(x_{i}))$ is equal to 2

As a conclusion, by using the genetic algorithm, we can solve the optimization problems by improving the created generations with a very good speed. Although we give up a small amount of accuracy of the algorithm.


# Providing solutions for the development and improvement of the used project
Sure, here are some suggestions for improving a project that uses genetic algorithms for curve fitting:

1. **Hierarchical Genetic Algorithm**: A hierarchical genetic algorithm can be used for curve fitting with B-splines. This approach is based on a novel hierarchical gene structure for the chromosomal representation, which allows determining the number and location of the knots, and the B-spline coefficients automatically and simultaneously.

2. **Improved Genetic Algorithm**: An improved genetic algorithm can be used for curve surface fitting. This could be particularly useful if your project involves fitting surfaces rather than just curves.

3. **Hybrid Approaches**: Combining genetic algorithms with other optimization techniques can often lead to better results. For example, you could use a genetic algorithm to explore the search space broadly and then use a local search method to fine-tune the solutions.

4. **Parameter Tuning**: The performance of a genetic algorithm can often be improved significantly by tuning its parameters, such as population size, mutation rate, crossover rate, and selection method. You could experiment with different parameter settings or use techniques such as grid search or random search to find optimal or near-optimal settings.

5. **Parallelization**: Genetic algorithms are inherently parallelizable because the fitness of each individual in the population can be evaluated independently. If your project involves large-scale problems or requires high accuracy, you could consider implementing a parallel version of the genetic algorithm to speed up the computations.

6. **Fitness Function**: The choice of fitness function is crucial in genetic algorithms. If your current fitness function does not provide satisfactory results, you could consider using a different fitness function or modifying the existing one.



# Questions


## 1.What problems do a very small or very large initial population cause?
#### Large initial population
- Advantages: the diversity of the initial population and the fact that we do not consider only special cases and have higher accuracy
- Disadvantages: Our iterations take more time. Our runtime increases and also we have more cost to check large population.
#### Small initial population
- Advantages: creating a new population is faster, and each iteration takes less time.
- Disadvantages: the selected chromosomes may all be special and good chromosomes may not be selected. Our accuracy is less and we have to do more iterations to find the answer.

## 2. If the number of population increases in each period, what effect will it have on the accuracy and speed of the algorithm?
There are several effects:

- Accuracy: A larger population size can potentially increase the accuracy of the algorithm. This is because a larger population can represent a greater diversity of solutions, increasing the likelihood of finding an optimal or near-optimal solution.

- Speed: Increasing the population size will likely slow down the algorithm. This is because a larger population requires more computational resources for operations such as selection, crossover, and mutation. Each individual in the population needs to be evaluated for its fitness so a larger population means more evaluations per generation

- Convergence: A larger population size might slow down the convergence of the algorithm. This is because with more individuals in the population, it might take more generations for good solutions to dominate the population

- Diversity: A larger population can maintain a higher level of genetic diversity for a longer period of time. This can be beneficial in preventing premature convergence to sub-optimal solutions

## 3. State and compare the impact of each of the crossover and mutation operations. Can be just one of them used ? Why ?
#### mutation
- It changes chromosomes randomly. We use a probability and according to it we change each of the genes of a chromosome. It prevents bias. We cannot use mutation alone because it has nothing to do with the initial population and alone cannot give us the answer
#### crossover
- Using 2 parents, we produce a new chromosome that inherits the characteristics of its parent. The drawback of using crossover alone is that it may not be able to solve the problem on its own, because in the train set, we do not have all the features of the original set and we need mutation to produce new features.


## 4. In your opinion, what solutions are there to get a faster solution to this particular problem?
One way we can prevent the code from running too much is to stop the code as soon as it reaches a zero SSE value. Also, by changing hyperparameters, we can make the speed of reaching the answer faster.

If we want to name the acceleration methods more precisely, the following can be mentioned:
- Limit the mutation: For faster convergence, you could limit the mutation. For example, when mutating, choose a new value with 20% probability between min and max and with 80% probability a random factor between 0.8 and 1.2 with which you multiply the old value.

- Optimize the gene pool: Randomly iterate a group of samples over a target function with the intent of achieving the lowest target value it is may be used to improve the convergence rate.

- Parallelization: Genetic algorithms are inherently parallel as each individual in the population can be evaluated independently

- Elitism: This is a method where a small proportion of the fittest individuals are guaranteed to survive to the next generation. This can speed up convergence as it ensures that good solutions are not lost from one generation to the next.

- Proper selection of individuals: The selection process plays a crucial role in genetic algorithms. Techniques like tournament selection, roulette wheel selection, or rank selection can be used effectively to speed up convergence.

At the end again the most important thing is to use good hyperparameters

## 5. Despite the use of these methods, it is still possible that the chromosomes will not change after several steps. Explain the reason for this and the problems it causes. What do you suggest to solve it? (Stagnation)
Stagnation is a common problem in genetic algorithms, which means that the population does not improve or change significantly over several generations. This can happen for several reasons:

- The population has reached a local optimum, which is a good but not the best solution, and cannot escape from it so we can use more mutation rate and more difference between them.
- The population has lost diversity, which means that most individuals are similar or identical, and there is not enough variation to explore new solutions.
- The population has converged prematurely, which means that the selection pressure is too high and eliminates potentially good solutions before they have a chance to evolve.

Stagnation can cause several problems for genetic algorithms such as:

- The algorithm may fail to find the global optimum, which is the best possible solution, or even a satisfactory solution.
- The algorithm may waste computational resources and time by repeating the same or similar evaluations without any progress.
- The algorithm may lose robustness and adaptability, which means that it cannot cope with changes in the problem or the environment.

There are several possible solutions to overcome stagnation in genetic algorithms, such as:

- Introducing diversity mechanisms, such as mutation, crossover, immigration, or niching, which can create new and different individuals in the population.
- Adjusting the parameters of the algorithm, such as population size, selection method, crossover rate, or mutation rate, which can affect the balance between exploration and exploitation of the search space.
- Using hybrid or adaptive algorithms, which can combine genetic algorithms with other optimization techniques or change their behavior according to the feedback from the evaluation function.

## 6.What solution do you suggest to end the program if the problem is not answered?
Here are some common termination conditions that you can consider:

- No improvement: Terminate the algorithm when there has been no improvement in the population for a certain number of iterations. For example, you can keep a counter which keeps track of the generations for which there has been no improvement in the population. Each time you don’t generate off-springs which are better than the individuals in the population, increment the counter.if the fitness of any of the off-springs is better then reset the counter to zero. The algorithm terminates when the counter reaches a predetermined value.

- Maximum number of generations: Terminate the algorithm when it reaches an absolute number of generations.

- Objective function value: Terminate the algorithm when the objective function value has reached a certain pre-defined value.

- Threshold fitness solution: The algorithm will terminate after the threshold fitness solution has been attained.

## 7. As the degree of polynomials increases, how does the time to find the coefficients change?
the time complexity can increase as the degree of the polynomial increases.

- Increased Search Space: As the degree of the polynomial increases, the number of coefficients that need to be determined also increases. This enlarges the search space, which can lead to an increase in the time required to find an optimal or near-optimal solution.

- Increased Complexity of Fitness Evaluation: The fitness of an individual in a genetic algorithm is typically determined by how well the polynomial it represents fits the data. As the degree of the polynomial increases, evaluating this fitness can become more computationally intensive, which can increase the time required per generation

- Increased Risk of Overfitting: Higher-degree polynomials can lead to overfitting where the model fits the training data too closely and performs poorly on new data

- Increased Number of Generations: If the increased complexity leads to slower convergence of the genetic algorithm more generations might be needed to reach an optimal or near-optimal solution

## 8. In your opinion, how does increasing or decreasing the number of points affect the implementation of the algorithm?
The number of data points can significantly affect the performance and results 
- Increased Number of Points: More data points can provide a more detailed representation of the underlying function that the algorithm is trying to fit. This can potentially lead to more accurate and reliable results. However, it can also increase the computational complexity, as the fitness of each individual (which typically involves comparing the predicted and actual values for each data point) needs to be evaluated

- Decreased Number of Points: Fewer data points can make the algorithm run faster as there are fewer fitness evaluations to perform it might also result in a less accurate representation of the underlying function.

it’s important to find a balance between the number of data points and the computational resources available. It might also be beneficial to use techniques such as cross-validation or bootstrapping to assess the reliability of the results and prevent overfitting

# Resources used

https://www.tutorialspoint.com/genetic_algorithms/genetic_algorithms_termination_condition.html

https://rdrr.io/cran/gafit/man/gafit.html

https://www.eng.auburn.edu/~smithae/files/A%20genetic%20algorithm%20approach%20to%20curve%20fitting.pdf

https://yoursageinformation.com/how-does-population-size-affect-genetic-algorithm/

https://annals-csis.org/Volume_1/pliks/167.pdf

https://codereview.stackexchange.com/questions/262963/genetic-algorithm-to-guess-coefficient-of-a-polynomial

https://cs.stackexchange.com/questions/116643/what-is-the-most-efficient-algorithm-to-compute-polynomial-coefficients-from-its

https://pubs.rsc.org/en/content/articlelanding/2008/cp/b807326k

https://search.r-project.org/CRAN/refmans/gafit/html/gafit.html